In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import shap
#import seaborn as sns

C:\Users\eee\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Normalize population data

In [606]:
tpu_count = pd.read_csv('F:\python\TPU_data\covid_TPU_count_selected_with_area.csv', index_col=0)

In [607]:
features = tpu_count.columns[1:46].values
print(features)

['Male' 'Female' '< 15 ' '15 - 24' '25 - 44' '45 - 64' '65+ '
 'Public rental housing' 'Subsidised home ownership housing'
 'Private permanent housing' 'Non-domestic housing' 'Temporary housing'
 '1' '2' '3' '4' '5' '6 and over' 'Without exclusively used rooms'
 'Less than 6,000' '6,000 - 9,999' '10,000 - 19,999' '20,000 - 29,999'
 '30,000 - 39,999' '40,000 - 59,999' '60,000 and over'
 'Work in the same district(5)'
 'Work in another district on Hong Kong Island'
 'Work in another district in Kowloon'
 'Work in another district in new towns'
 'Work in another district in other areas in the New Territories'
 'No fixed place/ Marine' 'Work at home' 'Places outside Hong Kong'
 'Manufacturing' 'Construction'
 'Import/export, wholesale and retail trades'
 'Transportation, storage, postal and courier services'
 'Accommodation and food services' 'Information and communications'
 'Financing and insurance'
 'Real estate, professional and business services'
 'Public administration, education, hu

In [608]:
tpu_count[features] = tpu_count[features].replace('-',int(0))

In [ ]:
for feature in features:
    print(i)
    tpu_count[i] = tpu_count[i].astype(int)
    tpu_count[i + '_ratio'] = tpu_count[i] / tpu_count['Total Population']

In [21]:
tpu_count['Population Density'] = tpu_count['Total Population'] / tpu_count['area']
tpu_count['Vulnerable population ratio'] = (tpu_count['< 15 '] + tpu_count['65+ ']) / tpu_count['Total Population']

In [22]:
tpu_count.to_csv('F:\python\TPU_data\covid_TPU_count_selected_normalized.csv')

## Select imported cases

In [3]:
df = pd.read_csv('covid_case_TPU_selected.csv', index_col=0)

In [696]:
print(df.shape)

(10603, 56)


In [697]:
#local_case = df.loc[df['Travel history']=='Nil',:]
local_case = df.loc[df['Travel history'].str.contains('Nil|NIl'),:]
#print(local_case)
travel_case = df.drop(local_case.index)
print(travel_case)

                Age     Sex   Case status        Onset date  \
Case no.                                                      
169       34.000000    male     Confirmed   2020/3/16 00:00   
170       36.000000    male     Confirmed   2020/3/14 00:00   
171       27.000000    male     Confirmed   2020/3/17 00:00   
172       28.000000    male     Confirmed   2020/3/15 00:00   
173       55.000000    male     Confirmed   2020/3/17 00:00   
174       31.000000    male     Confirmed    2020/3/9 00:00   
175       29.000000    male     Confirmed   2020/3/13 00:00   
176       29.000000  female     Confirmed   2020/3/15 00:00   
177       28.000000  female     Confirmed   2020/3/16 00:00   
178       29.000000    male     Confirmed   2020/3/16 00:00   
179       63.000000    male     Confirmed   2020/3/16 00:00   
180       38.000000    male     Confirmed   2020/3/16 00:00   
181       63.000000  female     Confirmed   2020/3/15 00:00   
183       24.000000  female     Confirmed   2020/3/12 0

In [6]:
df.loc[(df['Travel history']!='Nil')&(df['Remark I']=='Nil'),:]

,Age,Sex,Case status,Onset date,Residence,Travel history,Remark I,Remark II,Address,TPU,...,Construction,"Import/export, wholesale and retail trades","Transportation, storage, postal and courier services",Accommodation and food services,Information and communications,Financing and insurance,"Real estate, professional and business services","Public administration, education, human health and social work activities",Miscellaneous social and personal services,Others(8)
Case no.,,,,,,,,,,,,,,,,,,,,,
267,30.0,female,Confirmed,2020/3/13 00:00,189 Queen's Road West,Lives in US before March 16,Nil,NaN,189 Queen's Road West,113.0,...,363,2876.0,795.0,987,738,1768.0,1984.0,1650,1555.0,54
270,35.0,female,Confirmed,2020/3/17 00:00,"Phase 1, Mei Foo Sun Chuen, Lai Chi Kok",March 7 to 15: UK,Nil,NaN,"Phase 1, Mei Foo Sun Chuen, Lai Chi Kok",260.0,...,848,4406.0,1318.0,609,1171,2599.0,2798.0,4099,4687.0,60
271,51.0,female,Confirmed,2020/3/18 00:00,"Tower 12, Caribbean Coast, Tung Chung",March 5 to 10: US,Nil,NaN,"Tower 12, Caribbean Coast, Tung Chung",950.0,...,3259,7345.0,8797.0,4419,1767,2716.0,5881.0,4659,4178.0,129
272,11.0,male,Confirmed,2020/3/18 00:00,"Block 4, Prosperous Garden, Yau Ma Tei",March 7 to 16: UK,Nil,NaN,"Block 4, Prosperous Garden, Yau Ma Tei",225.0,...,3450,4690.0,1605.0,5139,734,1179.0,3552.0,2559,2286.0,54
273,36.0,male,Confirmed,2020/3/6 00:00,"Lam House, Hoi Fu Court, Yau Ma Tei","March 5 to 12: Belgium, Netherlands and Thailand",Nil,NaN,"Lam House, Hoi Fu Court, Yau Ma Tei",253.0,...,1312,3578.0,1054.0,1411,876,1748.0,2309.0,2446,2336.0,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2049,67.0,male,Confirmed,2020/7/17 00:00,"Block 19, Classical Gardens, Tai Po",Arrived at Hong Kong from the USA,Nil,NaN,"Block 19, Classical Gardens, Tai Po",725.0,...,249,573.0,232.0,148,87,244.0,286.0,392,604.0,53
2088,62.0,male,Confirmed,2020/7/17 00:00,"Block 1, Seaview Crescent, Tung Chung",Arrived at Hong Kong from the USA,Nil,NaN,"Block 1, Seaview Crescent, Tung Chung",950.0,...,3259,7345.0,8797.0,4419,1767,2716.0,5881.0,4659,4178.0,129
2324,20.0,male,Confirmed,2020/7/23 00:00,"Dorsett Tsuen Wan Hong Kong, Kwai Chung",Arrived at Hong Kong from India,Nil,NaN,"Dorsett Tsuen Wan Hong Kong, Kwai Chung",326.0,...,9641,15160.0,9976.0,8739,1945,2330.0,11417.0,11012,5717.0,385


In [698]:
print(local_case.shape)
print(travel_case.shape)

(9240, 56)
(1361, 56)


Let 'case_type' column indicate the case is a local/imported/unknwn case.

In [699]:
df['case type'] = 0

### cases without travel history

In [700]:
case_type = local_case['Remark I'].unique()
case_type = pd.DataFrame(case_type)
print(case_type)

names=['all_types']
case_type.columns=names
case_type.columns

                                                     0
0             Epidemiologically linked with local case
1    Epidemiologically linked with possibly local case
2                                           Local case
3                                                  Nil
4    Wedding banquet; Has epidemiological linkage t...
5    Wedding banquet; Has epidemiological linkage t...
6                                Visited Lan Kwai Fong
7    Mount Kelly School Hong Kong Cluster (cases 17...
8                            Close contact of case 191
9    Close contact of case 143 ; Under quarantine i...
10               Home cluster (cases 237, 303 and 304)
11               Home cluster (cases 154, 228 and 331)
12           Has epidemiological linkage with case 291
13                                     Son of case 221
14   Has epidemiological linkage with cases 266 and...
15           Has epidemiological linkage with case 195
16                   Close contact of case 229 and 338
17   Has e

Index(['all_types'], dtype='object')

In [701]:
#no travel history, but be linked with some confirmed cases
#Imported-related case should be local case, instead of imported case
local_type = case_type[case_type.all_types.str.contains('linkage|linked|local|Flight Attendant|Visited|son|contact|driver|nurse|Manager|Imported-related|Import-related|nil|pending',case=False)].reset_index()
local_type = local_type.drop(labels=local_type[local_type.all_types.str.contains('quarantine|transfer|studies|port ',case=False)].index).reset_index(drop=True)
case_type = case_type.drop(labels=local_type['index'])
print(local_type)
#print(local_type['all_types'].values)

     index                                          all_types
0        0           Epidemiologically linked with local case
1        1  Epidemiologically linked with possibly local case
2        2                                         Local case
3        3                                                Nil
4        4  Wedding banquet; Has epidemiological linkage t...
5        5  Wedding banquet; Has epidemiological linkage t...
6        6                              Visited Lan Kwai Fong
7        8                          Close contact of case 191
8       12          Has epidemiological linkage with case 291
9       13                                    Son of case 221
10      14  Has epidemiological linkage with cases 266 and...
11      15          Has epidemiological linkage with case 195
12      16                  Close contact of case 229 and 338
13      17  Has epidemiological linkage with cases 266 and...
14      18          Has epidemiological linkage with case 243
15      

In [702]:
cluster = case_type[case_type.all_types.str.contains('cluster',case=False)].reset_index()
#print(cluster['all_types'].values)
local_type = pd.concat([local_type, cluster],join='inner').reset_index(drop=True)
print(local_type)
#print(local_type['all_types'].values)
#print(type(local_type))

     index                                          all_types
0        0           Epidemiologically linked with local case
1        1  Epidemiologically linked with possibly local case
2        2                                         Local case
3        3                                                Nil
4        4  Wedding banquet; Has epidemiological linkage t...
5        5  Wedding banquet; Has epidemiological linkage t...
6        6                              Visited Lan Kwai Fong
7        8                          Close contact of case 191
8       12          Has epidemiological linkage with case 291
9       13                                    Son of case 221
10      14  Has epidemiological linkage with cases 266 and...
11      15          Has epidemiological linkage with case 195
12      16                  Close contact of case 229 and 338
13      17  Has epidemiological linkage with cases 266 and...
14      18          Has epidemiological linkage with case 243
15      

In [703]:
case_type = case_type.drop(labels=cluster['index'])

In [478]:
'''
not_known = case_type[case_type.all_types.str.contains('nil|pending',case=False)].reset_index()
print(not_known)
#print(not_known['all_types'].values)

case_type = case_type.drop(labels=not_known['index'])
'''

   index all_types
0      3       Nil
1    562  Pending#
2    564  Pending%


In [704]:
#only one case with no travel history, bu be classified into imported case
imported_type = case_type[case_type.all_types.str.contains('imported case|Studies|quarantine',case=False)].reset_index()
imported_type = imported_type.drop(labels=imported_type[imported_type.all_types.str.contains('linkage|linked|contact|transfer',case=False)].index).reset_index(drop=True)
print(imported_type['all_types'].values)

['Imported case']


In [705]:
#all left 'Remark I' without travel history but be linkd with other people can be classified into 'local case'
left_type = case_type.drop(labels=imported_type['index']).reset_index()
#print(case_type['all_types'].values)
print(left_type)
#local_type = local_type.append(left_type).reset_index()
#local_type = pd.concat([local_type, case_type])
#print(local_type)

    index                                          all_types
0       9  Close contact of case 143 ; Under quarantine i...
1      34  Transferred from Lei Yue Mun Park and Holiday ...
2      61  Has epidemiological linkage with case 338; Ref...
3      62  Has epidemiological linkage with case 719; Ref...
4      66  Has epidemiological linkage with case 411; Tra...
5      70  Has epidemiological linkage with case 759; Tra...
6      73  Has epidemiological linkage with case 833; Tra...
7      79  Has epidemiological linkage with cases 462, 75...
8      81  Has epidemiological linkage with case 902; Tra...
9      84  Transferred from quarantine centre; Has epidem...
10    112                 Transferred from quarantine centre
11    154  Transferred from quarantine centre; Has epidem...
12    167  Transferred from quarantine centre; Has epidem...
13    168  Transferred from quarantine centre; Has epidem...
14    187  Has epidemiological linkage with 1514, Transfe...
15    196  Has epidemiol

In [706]:
local_type = pd.concat([local_type, left_type],join='inner').reset_index(drop=True)
print(local_type)

     index                                          all_types
0        0           Epidemiologically linked with local case
1        1  Epidemiologically linked with possibly local case
2        2                                         Local case
3        3                                                Nil
4        4  Wedding banquet; Has epidemiological linkage t...
5        5  Wedding banquet; Has epidemiological linkage t...
6        6                              Visited Lan Kwai Fong
7        8                          Close contact of case 191
8       12          Has epidemiological linkage with case 291
9       13                                    Son of case 221
10      14  Has epidemiological linkage with cases 266 and...
11      15          Has epidemiological linkage with case 195
12      16                  Close contact of case 229 and 338
13      17  Has epidemiological linkage with cases 266 and...
14      18          Has epidemiological linkage with case 243
15      

Mark cases without travel history as 'local case' except for one case be marked as 'Imported case'

In [707]:
df.loc[(df['Travel history'].str.contains('Nil|NIl'))&(df['Remark I']=='Imported case'),'case type'] = 'imported case' 
df.loc[(df['Travel history'].str.contains('Nil|NIl'))&(df['Remark I']!='Imported case'),'case type'] = 'local case' 


### cases with travel history

In [708]:
case_type = travel_case['Remark I'].unique()
case_type = pd.DataFrame(case_type)
print(case_type)

names=['all_types']
case_type.columns=names
case_type.columns

                                                     0
0                                  Possibly local case
1                                        Imported case
2                                     Flight Attendant
3               Under quarantine in quarantine centre 
4                  Epidemiological linkage to case 218
5    Bolivia Tour Cluster; Has epidemiological link...
6    Bolivia Tour Cluster; Has epidemiological link...
7         Epidemiological linkage to cases 181 and 230
8                                    Studies in the UK
9                                                  Nil
10   Bolivia Tour Cluster; Has epidemiological link...
11   Mount Kelly School Hong Kong Cluster (cases 17...
12    Under compulsory home quarantine ; Studies in UK
13   Wedding cluster: Has epidemiological linkage t...
14                    Under compulsory home quarantine
15   Bolivia Tour Cluster; Has epidemiological link...
16   Hong Kong Sports Institute cluster (cases 172,...
17        

Index(['all_types'], dtype='object')

In [709]:
local_type = case_type[case_type.all_types.str.contains('local|Visited|son|driver|cluster',case=False)].reset_index()
local_type = local_type.drop(labels=local_type[local_type.all_types.str.contains('quarantine|transfer|studies|port ',case=False)].index).reset_index(drop=True)
local_type = local_type.drop(labels=local_type[local_type.all_types.str.contains('Bolivia Tour Cluster',case=False)].index).reset_index(drop=True)

case_type = case_type.drop(labels=local_type['index'])
print(local_type)

   index                                          all_types
0      0                                Possibly local case
1     11  Mount Kelly School Hong Kong Cluster (cases 17...
2     13  Wedding cluster: Has epidemiological linkage t...
3     16  Hong Kong Sports Institute cluster (cases 172,...
4     66       Family cluster (cases 467, 612, 613 and 622)
5    116           Epidemiologically linked with local case
6    117                                         Local case
7    124                                        Local case#


In [710]:
cluster = case_type[case_type.all_types.str.contains('Bar & Band Cluster',case=False)].reset_index()
#cluster = cluster.drop(labels=local_type[local_type.all_types.str.contains('Bolivia Tour Cluster',case=False)].index).reset_index(drop=True)
print(cluster['all_types'].values)
local_type = local_type.append(cluster).reset_index(drop=True)
print(local_type)
#print(local_type['all_types'].values)
#print(type(local_type))

['Bar & Band Cluster; Transferred from quarantine centre']
   index                                          all_types
0      0                                Possibly local case
1     11  Mount Kelly School Hong Kong Cluster (cases 17...
2     13  Wedding cluster: Has epidemiological linkage t...
3     16  Hong Kong Sports Institute cluster (cases 172,...
4     66       Family cluster (cases 467, 612, 613 and 622)
5    116           Epidemiologically linked with local case
6    117                                         Local case
7    124                                        Local case#
8     69  Bar & Band Cluster; Transferred from quarantin...


The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [711]:
case_type = case_type.drop(labels=cluster['index'])

In [487]:
'''
not_known = case_type[case_type.all_types.str.contains('nil|pending',case=False)].reset_index()
print(not_known)
#print(not_known['all_types'].values)

case_type = case_type.drop(labels=not_known['index'])
'''

   index                   all_types
0      9                         Nil
1    121  Pending for classification


In [712]:
imported_type = case_type[case_type.all_types.str.contains('imported case|Studies|quarantine|Flight|nil|pending|compulsory',case=False)].reset_index()
imported_type = imported_type.drop(labels=imported_type[imported_type.all_types.str.contains('linkage|linked|contact|transfer',case=False)].index).reset_index(drop=True)
Bolivia_Tour_Cluster = case_type[case_type.all_types.str.contains('Bolivia Tour Cluster',case=False)].reset_index()
#print(Bolivia_Tour_Cluster)
imported_type = imported_type.append(Bolivia_Tour_Cluster).reset_index(drop=True)
print(imported_type)
#print(imported_type['all_types'].values)

    index                                          all_types
0       1                                      Imported case
1       2                                   Flight Attendant
2       3             Under quarantine in quarantine centre 
3       8                                  Studies in the UK
4       9                                                Nil
5      12   Under compulsory home quarantine ; Studies in UK
6      14                   Under compulsory home quarantine
7      22                                   Flight attendant
8      24  Stay in hotel after arriving Hong Kong; Under ...
9      26                                     Flight Manager
10     32                                 Studies in the USA
11     34  Studies in UK; Under compulsory home quarantin...
12     35  Studies in Germany; Under compulsory home quar...
13     41  Studies in the UK; Under compulsory home quara...
14     43     Studies in UK Under compulsory home quarantine
15     44  Studies in No

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


Mark cases with travel history whose 'Remark I' is in local_type as 'local case';
Mark cases with travel history whose 'Remark I' is in imported_type as 'imported case'

In [713]:
for type in local_type['all_types'].unique():
    #print(index)
    df.loc[(~df['Travel history'].str.contains('Nil|NIl'))&(df['Remark I']== type),'case type'] = 'local case' 

for type in imported_type['all_types'].unique():
    #print(index)
    df.loc[(~df['Travel history'].str.contains('Nil|NIl'))&(df['Remark I']== type),'case type'] = 'imported case' 

In [714]:
case_type = case_type.drop(labels=imported_type['index'])

In [715]:
#print(case_type)
print(case_type['all_types'].values)

['Epidemiological linkage to case 218'
 'Epidemiological linkage to cases 181 and 230'
 'Has epidemiological linkage with case 169'
 'Studies in Switzerland; Has epidemiological linkage with case 198; Under compulsory home quarantine'
 'Has epidemiological linkage with case 241; case referred by Port Health Division'
 'Has epidemiological linkage with case 300; Under compulsory quarantine in hotel'
 'Has epidemiological linkage with case 207'
 'Has epidemiological linkage with case 191, 217 and 250'
 'Close contact of 229 and 337'
 'Has epidemiological linkage with case 351'
 'Has epidemiological linkage with case 350'
 'Wong Chuk Hang party (Has epidemiological linkage with cases 169, 208, 277 and 315)'
 'Flight attendant; Has epidemiological linkage with case 161'
 'Has epidemiological linkage with case 217'
 'Has epidemiological linkage with case 354'
 'Has epidemiological linkage with case 336 and 401'
 'Has epidemiological linkage with case 336 and 400'
 'Has epidemiological linka

Classify left 'Asymptomatic' cases without clear indication into 'unknown' class

In [716]:
df.loc[(~df['Travel history'].str.contains('Nil|NIl'))&(df['case type'] == 0)&(df['Case status'].str.contains('Asymptomatic|pending',case=False)), 'case type'] = 'unknown'

df.loc[(~df['Travel history'].str.contains('Nil|NIl'))&(df['case type'] == 0)&(df['Remark I'].str.contains('quarantine')), 'case type'] = 'imported case'

In [719]:
to_classify = pd.DataFrame(columns=df.columns)
to_classify = df.loc[(~df['Travel history'].str.contains('Nil|NIl'))&(df['case type'] == 0),:]
print(to_classify)

           Age     Sex Case status       Onset date  \
Case no.                                              
260       43.0    male   Confirmed  2020/3/19 00:00   
264       28.0    male   Confirmed  2020/3/17 00:00   
315       45.0    male   Confirmed  2020/3/20 00:00   
321       57.0  female   Confirmed  2020/3/20 00:00   
327       51.0  female   Confirmed  2020/3/20 00:00   
338       62.0    male   Confirmed  2020/3/11 00:00   
350       57.0  female   Confirmed  2020/3/18 00:00   
351       58.0    male   Confirmed  2020/3/14 00:00   
376       40.0  female   Confirmed  2020/3/20 00:00   
378       36.0  female   Confirmed  2020/3/21 00:00   
396       28.0    male   Confirmed  2020/3/19 00:00   
407       20.0  female   Confirmed  2020/3/17 00:00   
410       36.0    male   Confirmed  2020/3/17 00:00   
452       25.0    male   Confirmed  2020/3/14 00:00   
460       56.0  female   Confirmed  2020/3/24 00:00   
464       61.0    male   Confirmed  2020/3/20 00:00   
561       

In [720]:
to_classify.to_csv('F:\python\TPU_data\ccovid_case_TPU_selected_to_classify.csv')

Mannually classify the left 30 cases. (with travel history and linkage with others)

In [721]:
to_classify = pd.read_csv('F:\python\TPU_data\ccovid_case_TPU_selected_to_classify.csv', index_col=0)
print(to_classify)

          Age     Sex Case status      Onset date  \
Case no.                                            
260        43    male   Confirmed  2020/3/19 0:00   
264        28    male   Confirmed  2020/3/17 0:00   
315        45    male   Confirmed  2020/3/20 0:00   
321        57  female   Confirmed  2020/3/20 0:00   
327        51  female   Confirmed  2020/3/20 0:00   
338        62    male   Confirmed  2020/3/11 0:00   
350        57  female   Confirmed  2020/3/18 0:00   
351        58    male   Confirmed  2020/3/14 0:00   
376        40  female   Confirmed  2020/3/20 0:00   
378        36  female   Confirmed  2020/3/21 0:00   
396        28    male   Confirmed  2020/3/19 0:00   
407        20  female   Confirmed  2020/3/17 0:00   
410        36    male   Confirmed  2020/3/17 0:00   
452        25    male   Confirmed  2020/3/14 0:00   
460        56  female   Confirmed  2020/3/24 0:00   
464        61    male   Confirmed  2020/3/20 0:00   
561        67  female   Confirmed  2020/3/12 0

In [ ]:
for i in to_classify.index:
    #print(i) # 输出列名
    df.loc[i,'case type'] = to_classify.at[i,'case type']

In [747]:
df.loc[df['case type'] == 0,:]

,Age,Sex,Case status,Onset date,Residence,Travel history,Remark I,Remark II,Address,TPU,...,"Import/export, wholesale and retail trades","Transportation, storage, postal and courier services",Accommodation and food services,Information and communications,Financing and insurance,"Real estate, professional and business services","Public administration, education, human health and social work activities",Miscellaneous social and personal services,Others(8),case type
Case no.,,,,,,,,,,,,,,,,,,,,,


In [749]:
df.to_csv('F:\python\TPU_data\covid_case_TPU_selected_with_case_type.csv')

## Get the number of local/imported/unknown case in each TPU

In [797]:
df = pd.read_csv('F:\python\TPU_data\covid_case_TPU_selected_with_case_type.csv', index_col=0)

In [798]:
type_counts = df.groupby(['case type','TPU']).size().reset_index(name='counts')
print(type_counts)

         case type    TPU  counts
0    imported case  111.0       4
1    imported case  112.0     115
2    imported case  113.0      11
3    imported case  114.0      15
4    imported case  116.0       7
5    imported case  121.0       2
6    imported case  122.0       4
7    imported case  123.0       5
8    imported case  131.0      20
9    imported case  132.0      54
10   imported case  133.0       2
11   imported case  134.0       1
12   imported case  140.0       1
13   imported case  141.0       7
14   imported case  142.0      15
15   imported case  143.0      15
16   imported case  144.0      20
17   imported case  145.0       1
18   imported case  146.0      22
19   imported case  147.0       1
20   imported case  148.0       1
21   imported case  151.0      12
22   imported case  152.0       2
23   imported case  153.0      69
24   imported case  154.0       2
25   imported case  156.0       4
26   imported case  157.0       7
27   imported case  161.0       5
28   imported 

In [799]:
tpu_count = pd.read_csv('covid_TPU_count_selected_normalized.csv', index_col=0)
#print(tpu_count)

In [800]:
tpu_count['local case'] = 0
tpu_count['imported case'] = 0
tpu_count['unknown'] = 0

In [801]:
#print(type_counts.loc[(type_counts['case type'] == 'imported case') & (type_counts['TPU'] == 111), 'counts'].values)
for i in type_counts.loc[type_counts['case type'] == 'imported case','TPU']:
    #print(i)
    #print(tpu_count.loc[i, 'imported case'])
    tpu_count.loc[i, 'imported case'] = type_counts.loc[(type_counts['case type'] == 'imported case') & (type_counts['TPU'] == i), 'counts'].values

for i in type_counts.loc[type_counts['case type'] == 'local case','TPU']:
    tpu_count.loc[i, 'local case'] = type_counts.loc[(type_counts['case type'] == 'local case') & (type_counts['TPU'] == i), 'counts'].values

for i in type_counts.loc[type_counts['case type'] == 'unknown','TPU']:
    tpu_count.loc[i, 'unknown'] = type_counts.loc[(type_counts['case type'] == 'unknown') & (type_counts['TPU'] == i), 'counts'].values

### Get the ratio of Incidence Rate (only local case); 

In [802]:
tpu_count['ratio (local case / count)'] = tpu_count['local case']/tpu_count['count']
tpu_count['ratio (imported case / count)'] = tpu_count['imported case']/tpu_count['count']

In [803]:
tpu_count['ratio (local case / population)'] = tpu_count['local case']/tpu_count['Total Population']

In [807]:
tpu_count['ratio (local case / population)'].describe()

count    215.000000
mean       0.001176
std        0.001015
min        0.000000
25%        0.000627
50%        0.000969
75%        0.001431
max        0.008133
Name: ratio (local case / population), dtype: float64

In [806]:
tpu_count.to_csv('covid_TPU_count_selected_local.csv')